# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

In [128]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [129]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/opt/models")
sys.path.append("/opt/models/object_detection")

## Object detection imports
Here are the imports from the object detection module.

In [130]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [131]:
# What model to download.
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
dataPath = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/data'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(dataPath, 'spacenet_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [132]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

IOError: ('http error', 403, 'Forbidden', <httplib.HTTPMessage instance at 0x7fa43745d950>)

## Load a (frozen) Tensorflow model into memory.

In [133]:
#PATH_TO_CKPT = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/models/faster_rcnn_inception_resnet_v2_atrous/train/faster_rcnn_inception_50010_graph.pb'
PATH_TO_CKPT = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/models/faster_rcnn_inception_resnet_v2_atrous/train/output_inference_graph.pb'
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [134]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [135]:
from osgeo import gdal

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def load_image_into_numpy_array_Gdal(imagePath):
    # itegrate with matrix alegbra later
    #srcArray = gdalnumeric.LoadFile(image_path)
    #print(srcArray.shape)
    
    ds = gdal.Open(imagePath)
    band1 = np.array(ds.GetRasterBand(1).ReadAsArray())
    band1resize = 255*band1.astype(float)/np.max(band1)
    band2 = np.array(ds.GetRasterBand(2).ReadAsArray())
    band2resize = 255*band2.astype(float)/np.max(band2)
    band3 = np.array(ds.GetRasterBand(2).ReadAsArray())
    band3resize = 255*band3.astype(float)/np.max(band3)
    
    return np.array([band1resize, band2resize, band3resize]).transpose((1, 2, 0)).astype(np.uint8)
    

In [136]:
from osgeo import gdalnumeric
srcArray = gdalnumeric.LoadFile(image_path)
print(srcArray.shape)

ds = gdal.Open(image_path)
band1 = np.array(ds.GetRasterBand(1).ReadAsArray())
band1resize = 255*band1.astype(float)/np.max(band1)
band2 = np.array(ds.GetRasterBand(2).ReadAsArray())
band2resize = 255*band2.astype(float)/np.max(band2)
band3 = np.array(ds.GetRasterBand(2).ReadAsArray())
band3resize = 255*band3.astype(float)/np.max(band3)
print(np.array([band1resize, band2resize, band3resize]).shape)

#np.array(image.getdata()).shape

(3, 650, 650)
(3, 650, 650)


# Detection

In [137]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/annotations'#'test_images'
PATH_TO_TEST_IMAGES_DIR = '/data/spacenetV2_TestData/AOI_2_Vegas_Test/RGB-PanSharpen'
import glob
import numpy as np
numSamples = -1
TEST_IMAGE_PATHS_Total = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.tif'))
print(len(TEST_IMAGE_PATHS_Total))
if numSamples > -1:
    TEST_IMAGE_PATHS = np.random.choice(TEST_IMAGE_PATHS_Total, numSamples, replace=False)
else:
    TEST_IMAGE_PATHS = TEST_IMAGE_PATHS_Total
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

1282


In [139]:
from shapely.geometry import Polygon
import csv
from tqdm import tqdm
def makeRowPolygon(x):
    #xmin ymin, xmin ymax, xmax ymax, xmax, ymin
    poly = [Polygon([(x[1], x[0]), (x[1], x[2]), (x[3], x[2]), (x[3], x[2]), (x[3], x[0])])]
    return poly

def makeCSVEntry(boxes, image_np_shape, image_path, classes, scores, featureIdToReport=1, csvWriter=-1):
    im_width, im_height, image_depth = image_np_shape
    boxesPixel = np.squeeze(boxes)* np.transpose(np.array([im_height, im_width, im_height, im_width]))
    #Polygon([xmin, ymin], [xmin, ymax], [xmax, ymax])
    polygonList = np.apply_along_axis(makeRowPolygon, 1, boxesPixel)
    imageId = os.path.basename(image_path).split('_',1)[1]
    imageId = os.path.splitext(imageId)[0]
    buildingId = 0
    #print('classes')
    #print(classes)
    
    for polygon, score, featureId in zip(polygonList, np.squeeze(scores), np.squeeze(classes)):
        #print('polygon')
        #print(polygon)
        if featureIdToReport>-1 and featureId==featureIdToReport:
            buildingId = buildingId+1
            csvwriter.writerow([imageId, buildingId, polygon[0].wkt, score])
                
    if buildingId == 0:
        csvwriter.writerow([imageId, -1, 'POLYGON EMPTY', '0'])

    return 0


csvResultsFile = '/data/spacenetV2_TestData/AOI_2_Vegas_Test/resultsFasterRCNN_Vegas_shrink.csv'
with open(csvResultsFile, 'wb') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    
    with detection_graph.as_default():
      with tf.Session(graph=detection_graph) as sess:
        #print('tf.sessionPassed')
        for image_path in tqdm(TEST_IMAGE_PATHS):
          #image = Image.open(image_path)
          # the array based representation of the image will be used later in order to prepare the
          # result image with boxes and labels on it.
          #image_np = load_image_into_numpy_array(image)

          image_np = load_image_into_numpy_array_Gdal(image_path)
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
          image_np_expanded = np.expand_dims(image_np, axis=0)
          #print('preGetImageTensorByName')
          image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
          # Each box represents a part of the image where a particular object was detected.
          boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
          # Each score represent how level of confidence for each of the objects.
          # Score is shown on the result image, together with the class label.
          scores = detection_graph.get_tensor_by_name('detection_scores:0')
          classes = detection_graph.get_tensor_by_name('detection_classes:0')
          num_detections = detection_graph.get_tensor_by_name('num_detections:0')
          # Actual detection.
          (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
          makeCSVEntry(boxes, image_np.shape, image_path, classes, scores, featureIdToReport=1, csvWriter=csvwriter)
            
          #vis_util.visualize_boxes_and_labels_on_image_array(
          #    image_np,
          #    np.squeeze(boxes),
          #    np.squeeze(classes).astype(np.int32),
          #    np.squeeze(scores),
          #    category_index,
          #    use_normalized_coordinates=True,
          #    line_thickness=8)
          #plt.figure(figsize=IMAGE_SIZE)
          #plt.imshow(image_np)
          #print(np.squeeze(boxes))
          #print('scores')
          #print(scores)
          #print('classes')
          #print(classes)
          #print(num_detections)
      
    
    

100%|██████████| 1282/1282 [22:43<00:00,  1.06s/it]


In [113]:
image_np.shape

(650, 650, 3)

In [110]:
from shapely.geometry import Polygon
def makeRowPolygon(x):
    poly = [Polygon([(x[1], x[0]), (x[1], x[2]), (x[3], x[2])])]
    return poly

def makeCSVEntry(boxes, image_np_shape, image_path, classes, scores, featureIdToReport=1, csvWriter=-1)
        im_width, im_height, image_depth = image_np_shape
        boxesPixel = np.squeeze(boxes)* np.transpose(np.array([im_height, im_width, im_height, im_width]))
        #Polygon([xmin, ymin], [xmin, ymax], [xmax, ymax])
        polygonList = np.apply_along_axis(makeRowPolygon, 1, boxesPixel)
        imageId = os.path.basename(image_path).split('_',1)[1]
        buildingId = 0
        for polygon, score, featureId in zip(polygonList, scores, classes):
            if featureIdToReport>-1 && featureId==featureIdToReport:
                buildingId = buildingId+1
                csvwriter.writerow([imageId, buildingId, polygon.wkt, score])
                
        if buildingId == 0:
            csvwriter.writerow([imageId, -1, 'POLYGON EMPTY', '0'])
            


    

AttributeError: 'numpy.ndarray' object has no attribute 'wkt'

In [49]:
from shapely.geometry import Polygon
def createSpaceNetEntry(imageName, image, boxes, classes, scores):
    im_width, im_height = image.size
    boxesPixel = np.squeeze(boxes)* np.transpose(np.array([im_height, im_width, im_height, im_width]))
    
    np.apply_along_axis(boxesPixel.
    
    scores = np.squeeze(scores)                                                      
    

ImportError: No module named shapely

In [141]:
import io
jpeg = '/data/spacenetV2_TrainData/AOI_2_Vegas_Train_PASCALVOC/annotations/RGB-PanSharpen__-115.1794026_36.1686626998.jpg'
with tf.gfile.GFile(image_path, 'rb') as fid:
    encoded_jpg = fid.read()

encoded_jpg_io = io.BytesIO(encoded_jpg)

In [142]:
encoded_jpg_io